In [1]:
import pandas as pd
import numpy as np
import time                     #python 自带
from astropy.time import Time   #astropy 
from astropy import units as u 
from astropy.coordinates import cartesian_to_spherical


import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from moviepy.video.io.bindings import mplfig_to_npimage#动图生成
import moviepy.editor as mpy


def randUnitVec(number):
    phi      = np.random.rand(number,1) * 2 * np.pi  #方位角
    costheta = np.random.rand(number,1) * 2 - 1
    x = np.sqrt(1 - costheta**2) * np.cos(phi)
    y = np.sqrt(1 - costheta**2) * np.sin(phi)
    z = costheta
    r = np.concatenate([x, y, z], axis = 1)
    return r #(x,y,z)

def vecCross(a, b):
    e = np.zeros((3, 3, 3))
    e[0, 1, 2] = e[1, 2, 0] = e[2, 0, 1] = 1
    e[0, 2, 1] = e[2, 1, 0] = e[1, 0, 2] = -1

    c = [np.einsum("i,j,ijk->k", a[i,:],b[i,:], e) for i in range(0, len(a))]
    return np.vstack(c) #(x,y,z)

def vecDot(a, b):
    c = [np.inner(a[i],b[i]) for i in range(0, len(a))]
    return np.vstack(c) #(x,y,z)


def cart2spher(r):
    ans   =  cartesian_to_spherical(r[:,0], r[:,1], r[:,2])
    theta = ((np.pi/2 * u.radian - ans[1][0]) ).to('deg').value
    phi   = ans[2][0].to('deg').value
    return theta, phi


def transMat(ex, ey, ez):
    c = [np.vstack([ex[i], ey[i], ez[i]]) for i in range(0, len(ex))]
    return c


def W2ThetaPhi(trans, NVec):
    wt      = np.vstack([(np.inner(trans[i], NVec)).T for i in range(0, len(trans))])
    ewt     = wt / np.linalg.norm(wt, axis=1, keepdims=True)
    rLatLon = cartesian_to_spherical(ewt[:,0], ewt[:,1], ewt[:,2])
    theta   = np.pi/ 2 - rLatLon[1].value  #theta
    phi     = rLatLon[2].value # phi
    thetaPhi= np.vstack([theta, phi]).T
    return thetaPhi

def LNZ2Psi(LVec, NVec, ezd):
    psi = [np.arctan((vecDot(LVec, np.array([ezd[i]]) ) - vecDot(LVec, NVec) * vecDot(np.array([ezd[i]]), NVec)) \
                          / vecDot(NVec, vecCross(LVec, np.array([ezd[i]])))) for i in range(0, len(ezd))]
    return np.vstack(psi)


def Fplus1(theta, phi, psi):
    return 1/2 * (1 + (np.cos(theta))**2) * np.cos(2 * phi) * np.cos(2 * psi) \
- np.cos(theta)* np.sin(2 * phi) * np.sin(2 * psi) 

def Fcross1(theta, phi, psi):
    return 1/2 * (1 + (np.cos(theta))**2) * np.cos(2 * phi) * np.sin(2 * psi) \
+ np.cos(theta)* np.sin(2 * phi) * np.cos(2 * psi) 


def Fplus2(theta, phi, psi):
    return 1/2 * (1 + (np.cos(theta))**2) * np.sin(2 * phi) * np.cos(2 * psi) \
+ np.cos(theta)* np.cos(2 * phi) * np.sin(2 * psi) 

def Fcross2(theta, phi, psi):
    return 1/2 * (1 + (np.cos(theta))**2) * np.sin(2 * phi) * np.sin(2 * psi) \
- np.cos(theta)* np.cos(2 * phi) * np.cos(2 * psi) 


In [2]:
orbitDataTAIJI  =  pd.read_csv('orbitDataTAIJI.dat', index_col = 0)  #读取csv文件
orbitData      =  orbitDataTAIJI.loc[0:366,:]

sunPX = orbitData['sunPX']
sunPY = orbitData['sunPY']
sunPZ = orbitData['sunPZ']

earthPX = orbitData['earthPX']
earthPY = orbitData['earthPY']
earthPZ = orbitData['earthPZ']

scp1X = orbitData['scp1X']
scp1Y = orbitData['scp1Y']
scp1Z = orbitData['scp1Z']

scp2X = orbitData['scp2X']
scp2Y = orbitData['scp2Y']
scp2Z = orbitData['scp2Z']

scp3X = orbitData['scp3X']
scp3Y = orbitData['scp3Y']
scp3Z = orbitData['scp3Z']

scpX = (scp1X + scp2X + scp3X) / 3  
scpY = (scp1Y + scp2Y + scp3Y) / 3  
scpZ = (scp1Z + scp2Z + scp3Z) / 3  

scp = (np.vstack([scpX, scpY, scpZ])).T
scp1 = (np.vstack([scp1X, scp1Y, scp1Z])).T
scp2 = (np.vstack([scp2X, scp2Y, scp2Z])).T
scp3 = (np.vstack([scp3X, scp3Y, scp3Z])).T


d12 = scp2 - scp1 
d13 = scp3 - scp1
xd  = d12 + d13

ed12 = d12 / np.linalg.norm(d12, axis=1, keepdims=True)
ed13 = d13 / np.linalg.norm(d13, axis=1, keepdims=True)


exd = xd / np.linalg.norm(xd, axis=1, keepdims=True)
ezd = vecCross(ed13, ed12)
eyd = vecCross(ezd, exd)


trans = transMat(exd, eyd, ezd)


NVec = randUnitVec(1)
LVec = randUnitVec(1)
print(NVec, LVec)

# vecDot(ezd,eyd)
# vecDot(exd,eyd)
# vecDot(ezd,exd)

[[0.85415338 0.01490044 0.51980764]] [[-0.27411539  0.52447092  0.80609615]]


In [3]:
thetaPhi = W2ThetaPhi(trans, NVec)
theta = thetaPhi[:,0]
phi = thetaPhi[:,1]
psi = LNZ2Psi(LVec, NVec, ezd)[:,0]

####
Fp1 = Fplus1(theta, phi, psi)
Fc1 =  Fcross1(theta, phi, psi)
Fp2 = Fplus2(theta, phi, psi)
Fc2 =  Fcross2(theta, phi, psi)

In [4]:
ans = np.vstack([Fp1, Fc1, Fp2, Fc2]).T
ans

array([[-0.50923338, -0.85454576,  0.85455265, -0.50922152],
       [-0.48649153, -0.8656365 ,  0.8656451 , -0.48646829],
       [-0.46526157, -0.87487637,  0.87488593, -0.46522101],
       ...,
       [-0.57810843, -0.8135823 ,  0.81358417, -0.5781082 ],
       [-0.55210589, -0.83018935,  0.83019304, -0.55210339],
       [-0.52655559, -0.84504494,  0.84505073, -0.5265482 ]])